<a href="https://colab.research.google.com/github/JMacnar/ring_analysis/blob/master/sixmembered_rings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Six-membered ring analysis of small protein ligands

Easy to use analysis of six-membered ring in ligands present in prtoein complexes. For more details, checkout the [Ring analysis GitHub](https://github.com/jmacnar/ring_analysis.git) and read our manuscript:

[Macnar, J. M., Brzezinski, D., Chruszcz, M., & Gront, D. (2022). Analysis of protein structures containing HEPES and MES molecules. Protein Science, 31(9), e4415.](https://doi.org/10.1002/pro.4415) 

In [ ]:
#@title Install dependencies
%%bash -s 
set -e

#setup py3Dmol
if [ ! -f Py3Dmol_READY ]; then
  echo "Installing Py3Dmol ..."
  pip install py3Dmol 2>&1 1>/dev/null
  touch Py3Dmol_READY
fi
echo "Py3Dmol installed"

# setup conda
if [ ! -f CONDA_READY ]; then
  echo "Installing conda ..."
  wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh &>/dev/null
  chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
  bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local/ 2>&1 1>/dev/null
  conda install -y -q -c conda-forge py3dmol numpy 2>&1 1>/dev/null
  touch CONDA_READY
fi
echo "Conda installed"

#setup git
if [ ! -f GIT_READY ]; then
  git clone https://github.com/JMacnar/ring_analysis.git 2>&1 1>/dev/null
  cp ring_analysis/1ewk.pdb .
  touch GIT_READY
fi
echo "ring_analysis cloned"



In [ ]:
#@title Provide the three-letter PDB code of ligand
#@markdown If you would like to use the demo input left the "EPE" as the ligand's 3-letter code.
ligand_code = "EPE" #@param {type:"string"}
#@markdown The distance around a ligand used to visualize the structure of the ligand and its surroundings also can be provided
distance = 5 #@param {type:"integer"}

In [ ]:
#@title Upload PDB file
#@markdown If you would like to use the demo input just click "Cancel upload" and 1ewk will be used.

#@markdown Rember, "CONNECT" record must be present in yout pdb file to perform the analysis.

from google.colab import files
try:
  uploaded = files.upload()
  pdb_name = next(iter(uploaded))
except:
   pdb_name = "1ewk.pdb"
   print("No user's input provided, using 1ewk as an input.")



In [ ]:
#@title Run ring analysis
from IPython.utils import io
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()

with io.capture_output() as captured:
    !python ring_analysis/sixmembered_ring.py {pdb_name} {ligand_code} -d {distance}

data = captured.stdout
df = pd.DataFrame([x.split(';') for x in data.split('\n')[1:]], columns=[x for x in data.split('\n')[0].split(';')])
df= df.dropna()
df

In [ ]:
#@title Display 3D structure {run: "auto"}
import glob
import py3Dmol
color_protein = "B-factor" #@param ["B-factor", "rainbow"]
color_ligand = "white" #@param ["B-factor", "white"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
show_protein_surface = "No" #@param ["No", "White", "B-factor"]
show_ligand_surface = False #@param {type:"boolean"}
#@markdown Ligand surface is colored by B-factor
pdb_file = glob.glob(ligand_code+'-*-*-*.pdb')

protein = {'and':[{'resn':ligand_code,'invert':True}]}

def show_pdb(pdb_file1, show_sidechains=False, show_mainchains=False, show_prtoein_surface='No',
             show_ligand_surface=False, color_protein='rainbow', color_ligand='white'):
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file[0],'r').read(),'pdb')
  
  if color_protein == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color_protein == "B-factor":
    view.setStyle({'cartoon': {'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}}})
  view.addStyle({'line': {}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"TealCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"TealCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"TealCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"TealCarbon",'radius':0.3}})
  if show_protein_surface == 'B-factor':
    view.addSurface(py3Dmol.VDW,{'opacity':0.7,'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}},protein)
  elif show_protein_surface == 'White':
    view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'white'}, protein)
  
  if color_ligand == "B-factor":
    view.setStyle({'resn':ligand_code},{'stick':{'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}}})
  elif color_ligand == "white":
        view.setStyle({'resn':ligand_code},{'stick':{}})

  if show_ligand_surface:
    view.addSurface(py3Dmol.VDW,{'opacity':0.7,'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}},{'resn':ligand_code})
   
  
  view.zoomTo()
  return view

for pdb in pdb_file:
  print(pdb)
  show_pdb(pdb, show_sidechains, show_mainchains, show_protein_surface, show_ligand_surface, color_protein, color_ligand).show()

*A colab by Joanna Macnar and Ajajasja Ljubetič*

